In [1]:
import os
import sys
import re
import time
from pathlib import Path
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import warnings
import logging
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt

sys.path.append('../python')
warnings.filterwarnings('default')
logging.getLogger("pdfminer").setLevel(logging.ERROR)

DATA_DIR = Path('../../raw_data/cpc')

import api

rng = np.random.default_rng(1234)

/Users/ekung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ekung/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
meta_df = pd.read_csv("../../intermediate_data/cpc/meetings_metadata.csv")

In [3]:
for idx, row in meta_df.sample(30, random_state=rng).iterrows():
    year = row['year']
    date = row['date']
    with open(f"../../intermediate_data/cpc/{year}/{date}/agenda-item-summaries.txt", 'r') as f:
        text = f.read()
    with open(f"../../evals/agenda_item_summaries/{date}.txt", 'w') as f:
        f.write(text)

In [4]:
for idx, row in meta_df.sample(30, random_state=rng).iterrows():
    year = row['year']
    date = row['date']
    df = pd.read_pickle(f"../../intermediate_data/cpc/{year}/{date}/minutes-summaries.pkl")
    row2 = df.sample(1, random_state=rng).iloc[0]
    item_no = row2['item_no']
    title = row2['title']
    text = row2['response'].replace('---- YOUR RESPONSE FORMAT ----\n\n', '')
    text = f"ITEM NO. {item_no}\n{title}\n\n" + text
    with open(f"../../evals/minutes_summaries/{date}-{item_no}.txt", 'w') as f:
        f.write(text)

In [5]:
for idx, row in meta_df.sample(50, random_state=rng).iterrows():
    year = row['year']
    date = row['date']
    df = pd.read_pickle(f"../../intermediate_data/cpc/{year}/{date}/supplemental-docs-summaries.pkl")
    row2 = df.sample(1, random_state=rng).iloc[0]
    start_page = row2['start_page']
    end_page = row2['end_page']
    text = row2['response'].replace('==== YOUR RESPONSE FORMAT ====\n\n','')
    text = f"MEETING DATE: {date}\nSTART PAGE: {start_page}\nEND PAGE: {end_page}\n" + text
    with open(f"../../evals/supplemental_docs_summaries/{date}-{start_page}-{end_page}.txt", 'w') as f:
        f.write(text)